In [4]:
from scipy.optimize import minimize
from matplotlib import pyplot as plt
import numpy as np
import itertools
import copy
from DFNewmark import *
from DFFem import *

In [3]:
# Input 

E = 275.0*10**9     # Young's module (Pa)
rho = 2750.0        # Density (kg/m3)
Gc = 100.0          # Fracture energy (N/m)
stress_critical = 300.0*10**6   # Limit stress / critical stress (Pa)

A = 1*10**-3        # Cross sectional area (m2)
L = 1.05*10**-3     # Lenght of the bar (m)
x0 = 0              # Left extremitiy x coordinate / 0-initial
xf = L              # Rigth extremitiy x coordinate / f-final
n_el = 10           # Number of linear elements (n_el)
hun = L/n_el        # Size of the elemenets (h) for a uniform mesh (un)

strain_rate = 10.0**5   # Applied strain rate (s-1)
vel = strain_rate*L/2   # Applied velocity

# Set BC's

# Material id convention:
# 0 : line elemnt
# 1 : interface element
# 2 : Support left node
# 3 : Support right node
# 4 : Velocity applied left node
# 5 : Velocity applied right node
materials = [0] * n_el
materials.append(4)
materials.append(5)

# BC dictionary
bc_dict = {
    2: (0, "dirichlet"),
    4: (-vel, "velocity"),
    5: (vel, "velocity")
}

# Set time increment
dt_crit = 0.2*hun/((E/rho)**0.5)    # Critical time step
dt = dt_crit*0.4                    # Adopted time step (s)
time_simulation = 4.0*10**-7        # Total time of simulation (s)
n_steps = int(time_simulation/dt)   # Number of time-steps 

# Mesh 

# node_id[elem_index][local_node], returns the global node id
node_id = [[i,i+1] for i in range(n_el)]

# Identify each node has apllied BCs
# node_id.append([0])         # Support at left boundary
node_id.append([0])         # Applied velocity at left boundary
node_id.append([n_el])      # Applied velocity at right boundary


# Connect[el][j] returns the global index of local dof 'j' of the element 'el'
connect = copy.deepcopy(node_id)

# Number of degree of freedom
n_dofs = max(list(itertools.chain.from_iterable(connect))) + 1
n_points = n_dofs       # Number of points in the mesh

# Points coordinates for a uniform mesh
l = np.linspace(x0, xf, n_points)
node_coord = l

# Coordenates of integration points
x = [hun*i + hun*0.5 for i in range(n_el)]


# Set initial values

acel0 = np.zeros((n_dofs))          # Initial acceleration
p = np.zeros((n_steps+1, n_dofs))   # External forces
C = np.zeros((n_dofs, n_dofs))      # Damping

# Initial velocity (v0): velocity profile (vel) is a function v(x)
v0 = np.array([strain_rate*x for x in node_coord])

# Initial displacement
if strain_rate < 5.0 * 10.0**3:
    # Apply initial displacement neq zero to save computational time during pre-crack phase
    u0 = np.array([0.98*stress_critical*x / E for x in node_coord])
else:
    u0 = np.zeros((n_dofs))

# Initial damage 
d0 = [0., 0.04525, 0.086, 0.12225, 0.154, 0.18125, 0.204, 0.22225, 0.236, 0.24525, 0.25, 0.25025, 0.246, 0.23725, 
0.224, 0.20625, 0.184, 0.15725, 0.126, 0.09025, 0.05, 0.00525]


In [ ]:
def ElemLength(elem_index):
    """Returns the element length (hel)."""

    hel = node_coord[elem_index+1] - node_coord[elem_index]

    return hel

In [ ]:
def GlobalStiffness(E):
    
    

In [5]:
def GetStrain(u):
    # Total number of elements (linear + cohesive)
    numel = len(materials)
    # Initiation of variables
    strain = np.zeros(numel)
    for el in range(numel):

        if materials[el] == 0:
            # Strain[u,L] returns the strain value at each linear element 'el' 
            strain[el] = (u[connect[el][1]] - u[connect[el][0]]) / ElemLength(el)

    return strain

In [ ]:
l = 2.21*10**-6 # Lip-field regularization lenght
Yc = sigmac**2/(2*E)
lamb = 2*Yc*l/Gc
def h(d): return (2*d-d**2)/(1-d+lamb*d**2)**2

# Numerical integration on func
# Weight quadrature
w = 2.
def func(d): return w*sum([
    (0.5*(1-d[el])**2*DFMesh.E*epsilon_next[el]**2 + Yc*h(d[el]))*hel/2.
    for el in range(n_el)
])

def Ddash():

    ddash = minimize(
        fun=func,
        x0=dn,
        method='SLSQP',
        bounds=zip(dn, [1]*n_el),
        tol=1e-5,
    ).x

    return ddash


def UpperProjection(ddash):
    upper = [-minimize(
        lambda y: -ddash[np.searchsorted(coord, y[0])-1] + abs(x[el]-y[0])/l,
        x0=0.5*L,
        method='SLSQP',
        bounds=[(x0, xf)],
        tol=1e-5
    ).fun
        for el in range(n_el)
    ]

    return upper

def LowerPorjection():
    lower = [minimize(
        lambda y: ddash[np.searchsorted(coord, y[0])-1] + abs(x[el]-y[0])/l,
        x0=0.5*L,
        method='SLSQP',
        bounds=[(x0, xf)],
        tol=1e-5
    ).fun
        for el in range(n_el)
    ]

    return lower 



def Dlip():
    # Constraints for regularized damage for time step n+1 (dlip)
    const22a = [{'type': 'ineq', 'fun': lambda d:
            -(d[i] - d[i+1] - hel/l)} for i in range(0,n_el-2)]
    const22b = [{'type': 'ineq', 'fun': lambda d:
            -(d[i] - d[i-1] - hel/l)} for i in range(1,n_el-1)]
    const = tuple([*const22a,*const22b])


    # Regularized damage for time step n+1 (dlip) (Optimization 21b)
    dlip = minimize(
        fun=func,
        x0=dn,
        method='SLSQP',
        bounds=zip(dn, [1]*n_el),
        tol=1e-5,
        constraints=const
    ).x

    return dlip()

def PlotDamage(x, dn, dnext, ddash, upper, lower, dlip):
    fig, axes = plt.subplots()
    axes.grid(True, which='both')
    axes.axhline(y=0, color='k')

    plt.title(str("Damage field"))
    plt.xlabel(str("x"))
    plt.ylabel(str("D"))
    plt.plot(x, dn, label='dn')
    plt.plot(x, dnext, label='dnext')
    plt.plot(x, ddash, label='ddash')
    plt.plot(x, upper, label='upper')
    plt.plot(x, lower, label='lower')
    plt.plot(x, dlip, label='dlip')
    plt.legend()
    plt.show()

In [ ]:
# Main

# Initiation of variables
u = u0
v = v0
acel = acel0
d = d0

for n in range(n_steps):

    # Post process (stress, strain, energies)
    strain, stress, average_stress = GetStrain(u)
    
    # Get K, M and F
    M, F = GlobalSystem()

    # u,v,acel returns a vector for u,v and acel at every dof at the n step
    u, v, acel, d = Newmark_exp(M, u, v, acel, F, dt)


